# Initialize
This section is a developer aid; TODO remove later!
*Align with [storyboard Version 37](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=37)*

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "TODAY"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "10"  # TODO Change to '1' to test with default before push

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", ut.Server.usdf) # TODO try with "usdf" before push (else "summit")

In [4]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 5000  # YAGNI for Auto get more if this isn't enough to get all requested DAYS
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

# Experimental Features config
exp_tally = "true"  # default: true
exp_parsing = "true"  # default: true
experimental = dict()
experimental["exposure_type_tally"] = exp_tally == "true"
experimental["error_parsing"] = exp_parsing == "true"

In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=False,                # TODO change to False before push
    limit=limit,
)
allrep = AllReports(allsrc=allsrc)

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-10-24 to 2024-11-03 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac)
* [Summary plots of whole night](#Summary-plots-of-whole-night)
* [Time Accounting](#Time-Accounting)
* [Jira Tickets](#Jira-Tickets)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log)
* [Narrative Log](#Narrative-Log)
* [Developer Only Section](#Developer-Only-Section)

## Night log (summary) <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: AuxTel

```

```

Telescope Status: 

*Authors: *

```

```

...

*Authors: David Sanmartim, Carlos Morales, Kate Napier, Sandrine Thomas, Jacqueline Seron, Alysha Shugart, Guillem Megias Homar*

```
Most of the day was dedicated to the recovery of the TMA from the fault state reported in OBS-643. The telescope was pointing at an elevation of 87 degrees, with an azimuth cable wrap (ACW) limit switch activated. We could not reset its interlock due to EtherCAT communication issues. The azimuth cable wrap view in the EUI was blinking between its real position and the zero position, and the EIB azimuth system was in a fault state, preventing us from resetting its alarms or turning it on.

After some assessment from David Jimenez via Slack, we tried to reset EtherCAT from the NI Distributed System Manager, which worked for the TMA-PXI but not for the AXES-PXI. We then went to L8 and tried resetting them manually. We spent several hours trying different procedures (resetting the TMA-PXI, AXES-PXI, and EtherCAT) with assistance from David Jimenez and Felipe Daruich. We eventually eliminated the communication issues, but the azimuth limit switch was still triggered. Felipe suggested we should reset the TMA Pilz, which also required a new reset of the TMA-PXI. Overwriting the ACW limit switch and moving it ±3 degrees didn't work, and the limit switch was still active. Finally, the only way to get rid of it was by overwriting its limit and homing the azimuth itself.

After recovery was finished around 7:15 pm, we were finally able to start the system, finish FA tests (FA 330 failed twice and passed on the third try), and run the HP test. Since the temperature inside the dome and outside was very close, we were able to open the dome shutter and start AOS testing. 

AOS team report:
Tonight, we were able to start on sky earlier, around 10:30, allowing us to make substantial progress with testing. We began with a series of tests in which we perturbed each bending mode individually to observe its effects. Although we attempted to estimate the necessary corrections, the results initially seemed confusing. Toward the end, we realized that the M1M3 mirror expects absolute values rather than the relative values we were using, meaning we weren’t resetting the forces after each test. This oversight likely contributed to the increasing spherical aberration we observed tonight.
Beyond this, we also advanced in closing the loop. We captured some large, “giant” donut images that displayed a consistent signature across all frames. This may indicate an imperfection on M1M3.
We will continue the perturbation and closed-loop tests tomorrow.

AOS team wrapped up at 3, and it was decided to pass on the closed dome tests for tonight. Closed up. 

HVAC now set to daytime configuration.

2024-11-02 05:00 CLT Outside: 8.11 C, Inside: 11.35 C, cold: 1.69 C, general: 11.35 C
```

Confluence page: [testPlayer/BLOCK-R124](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testPlayer/BLOCK-R124)

Telescope Status: Telescope parked at El: 86.4
Az: -32, Dome: 328 
CSC states as requested

*Authors: David Sanmartim, Carlos Morales, Jacqueline Seron, Alysha Shugart, Guillem Megias Homar, Sandrine Thomas, Josh Meyers*

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-10-24**

,(times in UTC),
Moon Rise,2024-10-24 05:52:13.270,
Moon Set,2024-10-24 16:02:29.375,
Moon Illumination,52%,(% illuminated)
Astronomical Twilight (morning),2024-10-24 08:29:58.038,(-18 degrees)
Astronomical Twilight (evening),2024-10-24 00:23:57.671,(-18 degrees)
Solar Midnight,2024-10-24 04:26:58.413,
Nautical Twilight (morning),2024-10-24 09:00:20.402,(-12 degrees)
Nautical Twilight (evening),2024-10-23 23:53:34.933,(-12 degrees)
Civil Twilight (morning),2024-10-24 09:29:49.346,(-6 degrees)
Civil Twilight (evening),2024-10-23 23:24:05.527,(-6 degrees)


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

In [10]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
(Observing Operations - OBS)
(TODO: better tickets using Jira API)

In [11]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

- [OBS-407](https://rubinobs.atlassian.net/browse/OBS-407)

- [obs-407](https://rubinobs.atlassian.net/browse/obs-407)

- [OBS-630](https://rubinobs.atlassian.net/browse/OBS-630)

- [OBS-642](https://rubinobs.atlassian.net/browse/OBS-642)

- [OBS-498](https://rubinobs.atlassian.net/browse/OBS-498)

...

- [OBS-635](https://rubinobs.atlassian.net/browse/OBS-635)

- [OBS-626](https://rubinobs.atlassian.net/browse/OBS-626)

- [OBS-620](https://rubinobs.atlassian.net/browse/OBS-620)

- [OBS-643](https://rubinobs.atlassian.net/browse/OBS-643)

- [OBS-631](https://rubinobs.atlassian.net/browse/OBS-631)

In [12]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

#### Nightly Jira BLOCKs
- 20241101
    - [testPlayer/BLOCK-R124](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testPlayer/BLOCK-R124)
- 20241031
    - [testPlayer/BLOCK-R123](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testPlayer/BLOCK-R123)
- 20241030
    - [testPlayer/BLOCK-R122](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testPlayer/BLOCK-R122)
- 20241029
    - [testCycle/BLOCK-R121](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testCycle/BLOCK-R121)
- 20241028
    - [testCycle/BLOCK-R120](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testCycle/BLOCK-R120)

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
(TODO: BLOCKS observed) 

## Data Log <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

NOTE: *modified all_sources.py:uniform_field_counts()*
NOTE: Each tally count is now a link to a detailed table of exposures.

In [13]:
# Exposure Report
if experimental.get("exposure_type_tally"): 
    #  md("### Experimental Exposure field Tally ENABLED")
    md("Tally exposures with specific values of selected fields")
else:
    md("### Experimental Exposure field Tally DISASBLED")
    
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    if 0 == len(allsrc.exp_src.exposures[instrum]):
        continue

    # Facet counts
    if experimental.get("exposure_type_tally"): 
        counts = allsrc.uniform_exposure_field_counts(instrum)
        for fname, tally in counts.items():
            md(f"{fname}")
            df = pd.DataFrame.from_dict(tally, orient="index").T
            wdf = ut.wrap_dataframe_columns(df)
            display(HTML(wdf.to_html(render_links=True, escape=False)))

Tally exposures with specific values of selected fields

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 1424 total exposures**

science_program

...

,E2008,E1968,unknown,E1955,E1920,E1886,E1881,E1880,1880,BLOCK-T233,E1812,E1777,E1776,E1775,E1774,E1773,E1772,E1771,E1770,E1765,E1757,E1718,E1715,E1657
0,72,300,265,300,565,1728,1366,257,94,1788,257,29,29,29,29,29,29,29,29,1728,53,77,77,841


observation_type

,flat,bias,unknown,dark,engtest
0,4006,1982,5,4003,4


observation_reason

,sflat hi,bias,preimage,unknown,dark,flat,bot persistence,lambda,sflat lo,engtest,block-t233
0,648,191,60,5,1246,5623,342,18,75,4,1788


### (change) <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
#### TODO According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument. (that tally has been added below)
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [14]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

### Instrument: LSSTComCam (6)

* CC_O_20241101_000057 : 2024-11-02T04:31:43.789942
    - <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font>`Telescope was not tracking`

* CC_O_20241101_000058 : 2024-11-02T04:31:43.844342
    - <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font>`Telescope was not tracking`

* CC_O_20241101_000059 : 2024-11-02T04:31:43.881908
    - <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font>`Telescope was not tracking`

* CC_O_20241101_000089 : 2024-11-02T04:29:47.950494
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`Images not usable for Block-T253`

* CC_O_20241101_000090 : 2024-11-02T04:29:48.010969
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`Images not usable for Block-T253`

* CC_O_20241101_000091 : 2024-11-02T04:29:48.048521
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`Images not usable for Block-T253`

Note: *Added counts of Exposures with messages*

In [15]:
for instrum, recs in allsrc.exp_src.messages.items():
    md(f'**{instrum} {len(recs)} Exposures with messages**')
    if len(recs) > 0:
        df = allsrc.tally_exposure_flags(instrum)
        display(df.style.hide(axis="index"))

**LSSTComCam 6 Exposures with messages**

good,questionable,junk,none
0,0,3,3


## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [16]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-10-24 06:24:48.113248** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>To park the dome at the required position we did the following sequence twice: reset the dome cRIO, enable MTMount, home the dome and move the dome. In the last step, when moving the recurrent error (OBS-617) occurred. 


210809=Error of type Automation in subsystem AMCS: AFE is in fault. , 210809=Error of type Automation in subsystem AMCS: Azimuth drive nr.2 is in fault. , 210809=Error of type Automation in subsystem AMCS: Azimuth drive nr.3 is in fault. , 210809=Error of type Automation in subsystem AMCS: Azimuth drive nr.4 is in fault. , 210809=Error of type Automation in subsystem AMCS: Azimuth drive nr.5 is in fault.</pre>


- **2024-10-24 06:04:46.639911** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Finishing the BLOCK-194. None of the Camera Hexapods struts showed departing values in temperature.</pre>


- **2024-10-24 04:26:26.158072** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>At 01:23 local, we start one more time to recover the dome which faulted with the error reported in the OBS ticket OBS-617.</pre>


- **2024-10-24 03:49:46.462635** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Moving to BLOCK-194 Top end thermal tests: Soak Test w/ Camera Hexapod with ENABLED state between targets</pre>


- **2024-10-24 03:08:42.574830** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Work in progress related with the scheduler configuration deployment.</pre>


...

- **2024-11-02 17:33:55.708503** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>FA 409 passed on 2nd try. FA 330 failed again.


FA 330 passed on the third try.</pre>


- **2024-11-02 17:33:34.043316** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Actuators [330, 409] FAILED the bump test.


SAA (Single Actuator Axes) Failures:
  None
DAA (Dual Actuator Axes) Failures:
  - Actuator ID 330: Sec Index 80
  - Actuator ID 409: Pri Index 123</pre>


- **2024-11-02 15:31:20.238120** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We are putting the locking pin while people (Jeff B. and Victor K.) go to the dome area for a tour.</pre>


- **2024-11-02 15:28:55.846313** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We are now starting to turn on hexapods and rotator.</pre>


- **2024-11-02 15:28:12.224459** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We were able to move the telescope to zenith and remove operational limit switch that was triggered by just applying Elevation Parking settings.</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px"> REMOVE</font>
Contains stuff only expected to be useful to developers.

May also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [17]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **10 observing night(s)** with the last reported night starting on **2024-11-02 12:00:00**.
- min_dayobs='2024-10-24', max_dayobs='2024-11-03'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True
- Imported lsst.ts.logging_and_reporting from local packages.


## Data Status

In [18]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,13,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241024&max_day_obs=20241103
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241024&max_day_obs=20241103
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241024&max_day_obs=20241103
exposurelog/exposures.LSSTComCam,1424,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241024&max_day_obs=20241103
exposurelog/exposures.LSSTCam,10000,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=7500&limit=2500&min_day_obs=20241024&max_day_obs=20241103
exposurelog/messages,6,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241024&max_day_obs=20241103
narrativelog/messages,925,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-10-24T00%3A00%3A00&max_date_added=2024-11-03T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [19]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [20]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [13]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241024&max_day_obs=20241103

None
- Got 13 records.  


### Overview for Service: `exposurelog` [6]

...

None
- Got 6 records.  


### Overview for Service: `narrativelog` [925]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-10-24T00%3A00%3A00&max_date_added=2024-11-03T00%3A00%3A00

None
- Got 925 records.  


In [21]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

## Finale

In [22]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-11-02 21:30:54.576712
